# The polarization ellipse and Poincaré Sphere

### Code by Wasim Raja and Jennifer West

In [1]:
%matplotlib ipympl
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl 
import matplotlib.animation as animation
from matplotlib import gridspec
import scipy.ndimage
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d.axes3d import Axes3D
from mpl_toolkits.mplot3d.proj3d import proj_transform
import matplotlib.animation as anim
import pdb
#%matplotlib notebook



Bad key "text.kerning_factor" on line 4 in
/Users/jenniferwest/anaconda/envs/py37/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
class Arrow3D(FancyArrowPatch):

    def __init__(self, x, y, z, dx, dy, dz, *args, **kwargs):
        super().__init__((0, 0), (0, 0), *args, **kwargs)
        self._xyz = (x, y, z)
        self._dxdydz = (dx, dy, dz)

    def draw(self, renderer):
        x1, y1, z1 = self._xyz
        dx, dy, dz = self._dxdydz
        x2, y2, z2 = (x1 + dx, y1 + dy, z1 + dz)

        xs, ys, zs = proj_transform((x1, x2), (y1, y2), (z1, z2), self.axes.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))
        super().draw(renderer)
        
    def do_3d_projection(self, renderer=None):
        x1, y1, z1 = self._xyz
        dx, dy, dz = self._dxdydz
        x2, y2, z2 = (x1 + dx, y1 + dy, z1 + dz)

        xs, ys, zs = proj_transform((x1, x2), (y1, y2), (z1, z2), self.axes.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))

        return np.min(zs) 
def _arrow3D(ax, x, y, z, dx, dy, dz, *args, **kwargs):
    '''Add an 3d arrow to an `Axes3D` instance.'''

    arrow = Arrow3D(x, y, z, dx, dy, dz, *args, **kwargs)
    ax.add_artist(arrow)


setattr(Axes3D, 'arrow3D', _arrow3D)



In [3]:
mpl.rcParams['animation.ffmpeg_path'] = r'/usr/local/bin/ffmpeg'
writervideo = anim.FFMpegWriter(fps=10)

In [4]:
#constants
c = 3e8
z = 0

In [5]:
#inputs
phi_deg = 90 #phase difference between Ex and Ey
angle_th = 30 #pol position angle

phi = np.deg2rad(phi_deg)
angle_th = np.deg2rad(angle_th)

### Relations of variables
$\omega \times t $\
$\nu = \omega/2\pi $\
$T = 1/\nu $\
$T = 2\pi/\omega$

$k \times z$\
$Z = 2\pi/k$

$E_1 = E_0 cos(\theta)$\
$E_2 = E_0 sin(\theta)$

$\theta = tan^{-1}(E_2/E_1)$

$E_x = E_1sin(\omega t - k z)$\
$E_y = E_2sin(\omega t - k z + \phi)$

Fundamental quantities:

$E_0$ : amplitude\
$\theta$ : linear polarization angle\
$\phi$ : phase difference


In [6]:

E1 = np.cos(angle_th)
E2 = np.sin(angle_th)
coord_max = max(E1,E2)


omega = 10
nu = omega/(2*np.pi)
Lambda = c/nu
T = 1/nu # Time-Period of the wave
ntCycles=4
n_t = ntCycles*100
t_T = np.linspace(0,ntCycles*T,n_t)

k = 2*np.pi/Lambda
Z = 2*np.pi/k   # this is equal to 1-Lambda
nzCycles=4
n_z = nzCycles*100
z_Z = np.linspace(0,nzCycles*Z,n_z)


In [7]:
#z = z_Z
#t=t_T 

theta_t = []
mintheta=-np.pi
maxtheta=np.pi

thetas = np.random.uniform(low=mintheta, high=maxtheta,  size=(len(t_T),))
for i in range(0, len(t_T)):
    thetas[-1] = np.random.uniform(low=mintheta, high=maxtheta)
    thetas = np.roll(thetas, 1)
    theta_t.append(thetas)

### Wave in 3D as a function of time

In [8]:
#Set the viewing angle

el=30.
az=70.+90


#Set the parameters
showUnpolarized = False

showE1 = False
E1colour = "red"

showE2 = False
E2colour = "blue"

showSum = True
Etcolour = "green"

showSpokes = True
spokesOnly = False
showAxisLabels = False
rotateAxis = False

#Set the position of the arrows to draw
#position of arrows in units of wavelenghts
#max scale is 3.99 wavelengths
zwave1 = 0 
zwave2=3.99 

zloc1=int(zwave1*n_z/nzCycles)
zloc2=int(zwave2*n_z/nzCycles)  

#figS = plt.figure(figsize=(20,20))
figS = plt.figure(figsize=(8,8))
plt.tight_layout()

zmin = z_Z[0]
zmax = z_Z[n_z - 1]
w1=2
w2=3

ax = plt.axes(xlim=(-1, 1), ylim=(-1, 1), zlim=(t_T[0],t_T[n_t-1]), projection='3d')
line, = ax.plot3D([], [], [], lw=3, label="")

gs = gridspec.GridSpec(1, 1, width_ratios=[1]) 

ax3 = plt.subplot(gs[0], xlim=(-2, 2), ylim=(-2, 2), zlim=(zmin,zmax/zmax), projection='3d') 


centre3,= ax3.plot([],[],[], '-', lw=w1, alpha=1.0, c='black')

line1, = ax3.plot([],[],'-', lw=w1,c=E1colour, alpha=0.5, label = "Ex",)
line2, = ax3.plot([],[],'-', lw=w1,c=E2colour, alpha=0.5,label = "Ey",)

line3, = ax3.plot([],[],'-', lw=w1,c=Etcolour, alpha=0.5)


arrowa = ax3.quiver([],[],[],[],[],[])
arrowb = ax3.quiver([],[],[],[],[],[])

numSpokes = 48
Et_spoke_array = []
E1_spoke_array = []
E2_spoke_array = []

for iSpoke in range(0,numSpokes):
    Et_spoke_array.append(ax3.quiver([],[],[],[],[],[]))
    E1_spoke_array.append(ax3.quiver([],[],[],[],[],[]))
    E2_spoke_array.append(ax3.quiver([],[],[],[],[],[]))


Et = []

ax3.grid("off")
ax3.set_yticklabels([])
ax3.set_xticklabels([])
ax3.set_zticklabels([])
ax3.view_init(elev=el, azim=az)
# Get rid of colored axes planes
# First remove fill
ax3.xaxis.pane.fill = False
ax3.yaxis.pane.fill = False
ax3.zaxis.pane.fill = False

# Now set color to white (or whatever is "invisible")
ax3.xaxis.pane.set_edgecolor('w')
ax3.yaxis.pane.set_edgecolor('w')
ax3.zaxis.pane.set_edgecolor('w')

if showAxisLabels:
    ax3.set_xlabel("X")
    ax3.set_ylabel("Y")
    ax3.set_zlabel("Z")

def init():
    line.set_data([], [])
    return line,

def animatesum(i):
    global arrowa, arrowb, arrow, Et_spoke_array, E1_spoke_array, E2_spoke_array
    z = z_Z
    t=t_T[i] 
    
    if showUnpolarized:
        thetas = theta_t[i]
        E1 = np.cos(thetas)
        E2 = np.sin(thetas)
    else:
        E1 = np.cos(angle_th)
        E2 = np.sin(angle_th)

    Ex = E1*np.sin(omega*t - k*z)/coord_max
    Ey = E2*np.sin(omega*t - k*z + phi) /coord_max
    
    #Ex = E1*np.cos(omega*t - k*z)/coord_max
    #Ey = E2*np.cos(omega*t - k*z + phi) /coord_max
    
    #make an array of the amplitudes to plot later
    Et.append(np.sqrt(Ex[-1]**2+Ey[-1]**2))

    '''
    if showCircular:
        phase = (omega *t - k*z) 
        xL = E1*np.sin(-phase)/coord_max
        yL = E1*np.sin(-phase+np.pi/2)/coord_max
        xR = E2*np.sin(phase)/coord_max
        yR = E2*np.sin(phase+np.pi/2)/coord_max
    '''
    
    
    if showE1:
        if (spokesOnly!=True):
            line1.set_data(np.zeros(len(Ex)), Ey)
            line1.set_3d_properties(z/zmax)
    if showE2:
        if (spokesOnly!=True):
            line2.set_data(Ex, np.zeros(len(Ey)))
            line2.set_3d_properties(z/zmax)
    
    if showSum:
        line3.set_data(Ex, Ey)
        #divide by zmax to normalize z-values
        line3.set_3d_properties(z/zmax)
    
    centre3.set_data([0,0],[0,0])
    #normalized z-axis from 0 to 1 
    centre3.set_3d_properties([0,1])
    
    if showSum:
        arrowa.remove()
        arrowa = ax3.quiver(0,0,z_Z[zloc1]/zmax, Ex[zloc1],Ey[zloc1],0, arrow_length_ratio=0.0, lw=w2, color=Etcolour)
        arrowb.remove()
        arrowb = ax3.quiver(0,0,z_Z[zloc2]/zmax, Ex[zloc2],Ey[zloc2],0, arrow_length_ratio=0.0, lw=w2, color=Etcolour)
    
    if(showSpokes):
        spokes = np.linspace(0+1,n_z-2,len(Et_spoke_array))
        for s in range(0, len(Et_spoke_array)):
            if (showSum or spokesOnly):
                Et_spoke_array[s].remove()
            if showE1:
                E1_spoke_array[s].remove()
            if showE2:
                E2_spoke_array[s].remove()
            
            loc=int(spokes[s])
            if (showSum or spokesOnly):
                Et_spoke_array[s] = ax3.quiver(0,0,z_Z[loc]/zmax, Ex[loc],Ey[loc],0, arrow_length_ratio=0.05, lw=2, color=Etcolour,alpha=0.4)
            if showE1:
                E1_spoke_array[s] = ax3.quiver(0,0,z_Z[loc]/zmax, 0,Ey[loc],0, arrow_length_ratio=0.05, lw=2, color=E1colour,alpha=0.4)
            if showE2:
                E2_spoke_array[s] = ax3.quiver(0,0,z_Z[loc]/zmax, Ex[loc],0,0, arrow_length_ratio=0.05, lw=2, color=E2colour,alpha=0.4)
            
            #End Et vector
            Et_spoke_array[-1].remove()
            loc=int(spokes[-1])
            Et_spoke_array[-1] = ax3.quiver(0,0,z_Z[loc]/zmax, Ex[loc],Ey[loc],0, arrow_length_ratio=0.05, lw=3, color=Etcolour,alpha=1)
    if rotateAxis:
        ax3.view_init(elev=el, azim=i/2)
    return line3

# Get rid of colored axes planes
# First remove fill
ax3.xaxis.pane.fill = False
ax3.yaxis.pane.fill = False
ax3.zaxis.pane.fill = False

# Now set color to white (or whatever is "invisible")
ax3.xaxis.pane.set_edgecolor('w')
ax3.yaxis.pane.set_edgecolor('w')
ax3.zaxis.pane.set_edgecolor('w')

ax3.xaxis._axinfo["grid"].update({"color":(0.1,0.1,0.1,0.1)})
ax3.yaxis._axinfo["grid"].update({"color":(0.1,0.1,0.1,0.1)})
ax3.zaxis._axinfo["grid"].update({"color":(0.1,0.1,0.1,0.1)})


# Get rid of the grid as well:
ax3.grid(False)

animS = animation.FuncAnimation(figS, animatesum, init_func=init, frames = n_z, interval = 30, repeat = False)
#animS.save('elliptical_polarized_'+str(np.rad2deg(phi))+'_'+str(np.rad2deg(angle_th))+'.mp4', writer=writervideo)
plt.show()

FigureCanvasNbAgg()

In [ ]:
print(np.array(Et).shape)
fig = plt.figure()
plt.plot(t,Et[0:400])

### Polarization ellipse

In [ ]:
# Polarization ellipse changing in time
 
fig = plt.figure()  
axis = plt.axes(xlim =(-1.2, 1.2), 
                ylim =(-1.2, 1.2))  
axis.axis('square') 
linex, = axis.plot([], [], 'ro', markersize=3, label = "Ey", alpha=0.3)  
liney, = axis.plot([], [], 'bo', markersize=3, label = "Ex", alpha=0.3)  
line, = axis.plot([], [], 'go', markersize=3, label="Etotal", alpha=0.3) 

axis.set_xlabel("Ex(t)")
axis.set_ylabel("Ey(t)")
axis.legend()
# what will our line dataset 
# contain? 
def init():  
    line.set_data([], [])  
    return line,  

# initializing empty values 
# for x and y co-ordinates 
xdata, ydata = [], []  
x_const, y_const = [], []    
# animation function  
def animate(i):  
    # t is a parameter which varies 
    # with the frame number 
    t =  t_T[i] 

    # x, y values to be plotted  
    Ex = E1*np.sin(omega*t - k*z)/coord_max
    Ey = E2*np.sin(omega*t - k*z + phi) /coord_max

    # appending values to the previously  
    # empty x and y data holders  
    xdata.append(Ex)  
    ydata.append(Ey) 
    x_const.append(0)
    y_const.append(0)

    linex.set_data(x_const, ydata)
    liney.set_data(xdata, y_const)
    line.set_data(xdata, ydata)

    return line, 

# calling the animation function      
anim = animation.FuncAnimation(fig, animate, init_func = init,  
                               frames = t_T.shape[0], interval = 20, repeat = False)  

#anim.save('elliptical_polarized_'+str(np.rad2deg(phi))+'_'+str(np.rad2deg(angle_th))+'_with_ExEy_2D.mp4', writer=writervideo)
plt.show()


### Poincaré Sphere

In [ ]:
tau_0 = 0.5*np.arctan2(np.tan(2.0*angle_th)*np.cos(phi),1.0)
eps=1e-10
axis_ratio = 0
#The trick is to rotate the x-axis onto the MAJ-axis
#The Mathematics does not distinguish between x-axis 
#and y-axis or the maj-axis and the minor-axis .
if ((phi == 0.0) or (E1*E2 < eps)):
#Linear polarisation
    tau_0 = np.rad2deg(angle_th)
    epsilon_0 = 0.0
    s_maj_axis = 1.0
    s_min_axis = 0.0
elif(phi!=0.0):
    if(np.cos(phi) >= 0.0):
        if(angle_th <= np.pi/4.0):
            tau_0 = tau_0
        elif(angle_th > np.pi/4.0):
            tau_0 = np.pi/2.0 + tau_0 #tau becomes -ve in this case

        elif(np.cos(phi) < 0.0):
            if(angle_th > np.pi/4.0):
                tau_0 = tau_0 + np.pi/2.0
            elif(angle_th <= pi/4.0):
                tau_0 = np.pi + tau_0 #tau becomes -ve in this case

    a2 = ((np.sin(angle_th)*np.cos(angle_th)*np.sin(phi))**2)/((np.sin(tau_0)*np.cos(angle_th))**2 + (np.cos(tau_0)*np.sin(angle_th))**2 - 2.0*np.sin(tau_0)*np.cos(tau_0)*np.sin(angle_th)*np.cos(angle_th)*np.cos(phi))
    b2 = ((np.sin(angle_th)*np.cos(angle_th)*np.sin(phi))**2)/((np.cos(tau_0)*np.cos(angle_th))**2 + (np.sin(tau_0)*np.cos(angle_th))**2 + 2.0*np.sin(tau_0)*np.cos(tau_0)*np.sin(angle_th)*np.cos(angle_th)*np.cos(phi))

    a = np.sqrt(a2)
    b = np.sqrt(b2)
    tau_0 = np.rad2deg(tau_0)
    s_maj_axis = max(a,b)
    s_min_axis = min(a,b)
    if (s_min_axis <= eps):
        s_min_axis = eps

    if (phi > 0): # by convention +phi => LCP
        axis_ratio = s_maj_axis/s_min_axis   # -phi => RCP
    elif (phi < 0): # on the PS, NP = LCP
        axis_ratio = -s_maj_axis/s_min_axis  #SP = RCP
    epsilon_0 = np.rad2deg(np.arctan2(1,axis_ratio))

print("s_maj_axis = ",s_maj_axis)
print("s_min_axis = ",s_min_axis)
print("angle_th = ",angle_th)                                 
print("axis ratio = ",np.divide(s_maj_axis,s_min_axis))
print("E1 = ",E1)
print("E2 = ",E2)
print("E2/E1 = ",np.divide(E2,E1))
print("phi_deg = ",phi_deg)
print("tau_0 (linear PA) = ",tau_0) # 2*tau_0 = longitude in Poincare Sphere
print("epsilon_0 (latitude in Poincare Sphere) = ",epsilon_0)     # 2*epsilon_0 = latitude in Poincare Sphere 


In [ ]:
#********** Define the Poincare Sphere ************
R = 1
n_tau = 20
n_epsilon = 15 
tau = np.linspace(0,180,n_tau)          # 2*tau = longitude
tau = np.deg2rad(tau)

epsilon = np.linspace(-45,45,n_epsilon)   # 2*epsilon = latitude
epsilon = np.deg2rad(epsilon)
x_par = np.zeros((n_tau,n_epsilon ))
y_par = np.zeros((n_tau,n_epsilon ))
z_par = np.zeros((n_tau,n_epsilon ))
for j in range(0, n_epsilon):
    for i in range(0, n_tau):
        x_par[i,j] = R*np.cos(2*epsilon[j])*np.cos(2*tau[i])
        y_par[i,j] = R*np.cos(2*epsilon[j])*np.sin(2*tau[i])
        z_par[i,j] = R*np.sin(2*epsilon[j])
        
tau_0_rad = np.deg2rad(tau_0)
epsilon_0_rad = np.deg2rad(epsilon_0)
x_par_0 = R*np.cos(2*epsilon_0_rad)*np.cos(2*tau_0_rad)
y_par_0 = R*np.cos(2*epsilon_0_rad)*np.sin(2*tau_0_rad)
z_par_0 = R*np.sin(2*epsilon_0_rad)


In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(projection='3d')

ax.plot_wireframe(x_par, y_par, z_par,alpha=0.15)
x_eq = np.where(z_par==0,x_par,0)
y_eq = np.where(z_par==0,y_par,0)
z_eq = np.where(z_par==0,z_par,0)
ax.plot_surface(x_eq, y_eq, z_eq, rstride=1, cstride=1, alpha=0.05)

ax.scatter(0,0,0,c="k",s=100)

#ax.set_box_aspect(aspect=(1, 1, 1))

ax.arrow3D(0,0,0,x_par_0, y_par_0, z_par_0,color="r",mutation_scale=30,ec ='red',fc='red')

ax.set_xlabel("Stokes Q")
ax.set_ylabel("Stokes U")
ax.set_zlabel("Stokes V")

# Get rid of colored axes planes
# First remove fill
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False

# Now set color to white (or whatever is "invisible")
ax.xaxis.pane.set_edgecolor('w')
ax.yaxis.pane.set_edgecolor('w')
ax.zaxis.pane.set_edgecolor('w')

ax.xaxis._axinfo["grid"].update({"color":(0.1,0.1,0.1,0.1)})
ax.yaxis._axinfo["grid"].update({"color":(0.1,0.1,0.1,0.1)})
ax.zaxis._axinfo["grid"].update({"color":(0.1,0.1,0.1,0.1)})

plt.title("Poincaré Sphere")

# Get rid of the grid as well:
#ax.grid(False)



$\begin{pmatrix}
\hat {{e}_R} \\
\hat {{e}_L}
\end{pmatrix}$
=
$\begin{pmatrix}
1 & e^{i\pi/2} \\
e^{i\pi/2} & 1  
\end{pmatrix}$
$\begin{pmatrix}
\hat {{e}_x} \\
\hat {{e}_y}
\end{pmatrix}$

In [ ]:
L2C = ([(1,1j),(1j,1)])
EXY = ([Ex, Ey])
ERL = np.matmul(L2C, EXY)
rot = ([(np.cos(chi_rot),np.sin(chi_rot)),(-np.sin(chi_rot),np.cos(chi_rot))])
EXY_rot = np.matmul(rot, EXY)
Er = ERL[0]
El = ERL[1]
Ex_rot = EXY_rot[0][1:98]
Ey_rot = EXY_rot[1][1:98]